## FEATURE SELECTION - FootySage

En este cuaderno se define un flujo de trabajo para la selección y análisis de características relevantes utilizando los datos de *Open Data* de ***StatsBomb***. 

Este proceso comienza con la integración de información proveniente de diversas fuentes, como partidos, alineaciones y eventos, consolidándola en un dataset estructurado. Cada fila del dataset representa un partido, enriquecido con métricas calculadas que reflejan aspectos clave del rendimiento, como estadísticas agregadas de pases, disparos, recuperaciones, faltas, entre otros.

A partir de este dataset, se entrenarán modelos básicos de Machine Learning, como Árboles de Decisión (Decision Tree), Regresión Logística (Logistic Regression) y K-Nearest Neighbors (KNN). El objetivo es evaluar su capacidad de clasificación y obtener conclusiones sobre la influencia de las características seleccionadas. Este enfoque inicial proporcionará una base sólida para ampliar el análisis hacia conjuntos de datos más extensos e implementar modelos más avanzados en futuras fases del proyecto.

### Estudio de una competición en una temporada (Competición regular)

#### Carga de datos y preprocesamiento

En primer lugar seleccionamos la competición y la temporada de esa competición que queremos estudiar.

In [1]:
from src.fetch_data import get_competition_id_and_season_id

# seleccionamos el torneo a estudiar (competición, temporada y género)
competition_name = "La Liga"
competition_gender = "male"
season_name = "2015/2016"
season_name_for_filename = "2015_2016"

competition_id, season_id = get_competition_id_and_season_id(competition_name, competition_gender, season_name)
print(f"We are going to study the tournament {competition_name + ' (' + season_name + ', ' + competition_gender + ')'} whose competition_id={competition_id} and season_id={season_id}")

We are going to study the tournament La Liga (2015/2016, male) whose competition_id=11 and season_id=27


Dada esa competición vamos a obtener cuales son todos los partidos que se han jugado en tal competición esa temporada.

In [2]:
from src.fetch_data import get_matches

matches_df = get_matches(competition_id, season_id)
print(f"Number of matches in the competition that season: {matches_df.shape[0]}")

# comprobamos que todos los partidos estén disponibles para obtener información
column_name = "match_status"
if matches_df[column_name].nunique() == 1: 
    print(f"All values in column '{column_name}' are equal: {matches_df[column_name].iloc[0]}")
else:
    print(f"The values in column '{column_name}' are different.")

# ordenamos los partidos por semana de competición y hora de inicio para que un futuro cuando tengamos que buscar datos
# de partidos anteriores podamos ver a partir de que momento consultarlo
matches_sorted_by_week_df = matches_df.sort_values(by=["match_week","kick_off"])

Number of matches in the competition that season: 380
All values in column 'match_status' are equal: available


Tras tener todos los partidos de la competición ordenados a nuestro gusto, nos disponemos a obtener los datos relacionados con eventos, alineaciones... de cada partido. 

Estos datos son (si pongo '(x2)' es porque me refiero a el equipo de casa como el de fuera y 'x' en el nombre de la variable hace referencia a 'home' y 'away'):
- Estadísticas generales:
  - Número de tiros totales (x2) --> total_shots_x
  - Ratio de tiros a puerta (x2) --> shots_on_target_ratio_x
  - Distancia promedio a la portería desde donde se hacen los tiros (x2) --> average_shot_distance_x
  - Tiros con alto xG (umbral > 0.2) (x2) --> shots_high_xG_x
  - Número de tiros dentro del área (x2) --> shots_inside_area_x
  - Ratio de tiros dentro del área (x2) --> shots_inside_area_ratio_x
  - Número de tiros con el pie (derecho e izquierdo) (x2) --> shots_foot_x
  - Número de tiros con la cabeza (x2) --> shots_head_x
  - Número de tiros con otra parte del cuerpo (x2) --> shots_other_x

In [3]:
from src.preprocessing import *
import os


# procesamos todos los partido de la competición
matches_processed_df = process_all_matches(matches_sorted_by_week_df)

# Guardamos la información procesada en un csv por si en algún momento necesitamos volver a tener que cargar esta información
output_dir = "data/processed"
filename = f"{competition_name}({season_name_for_filename}_{competition_gender})_processed.csv"
output_path = os.path.join(output_dir, filename)
matches_processed_df.to_csv(output_path, index=False)
print(f"All matches from the competition processed and save in {output_dir}/{filename}")

All matches from the competition processed and save in data/processed/La Liga(2015_2016_male)_processed.csv


### PRUEBAS

In [66]:
matches_sorted_by_week_df.head()

,match_id,match_date,kick_off,competition,season,home_team,away_team,home_score,away_score,match_status,...,last_updated_360,match_week,competition_stage,stadium,referee,home_managers,away_managers,data_version,shot_fidelity_version,xy_fidelity_version
344,3825565,2015-08-22,18:30:00.000,Spain - La Liga,2015/2016,Espanyol,Getafe,1,0,available,...,None,1,Regular Season,RCDE Stadium,Jesús Gil Manzano,Sergio González Soriano,Francisco Escriba Segura,1.1.0,2,2
345,3825564,2015-08-22,18:30:00.000,Spain - La Liga,2015/2016,RC Deportivo La Coruña,Real Sociedad,0,0,available,...,None,1,Regular Season,Estadio Abanca-Riazor,Antonio Miguel Mateu Lahoz,Víctor Sánchez del Amo,David Moyes,1.1.0,2,2
358,266236,2015-08-23,18:30:00.000,Spain - La Liga,2015/2016,Athletic Club,Barcelona,0,1,available,...,2021-06-13T16:17:31.694,1,Regular Season,San Mamés Barria,NaN,Ernesto Valverde Tejedor,Luis Enrique Martínez García,1.1.0,2,2
193,3825570,2015-08-24,20:30:00.000,Spain - La Liga,2015/2016,Granada,Eibar,1,3,available,...,None,1,Regular Season,Estadio Nuevo Los Cármenes,Ignacio Iglesias Villanueva,José Ramón Sandoval Huertas,José Luis Mendilibar Etxebarria,1.1.0,2,2
194,3825562,2015-08-21,20:30:00.000,Spain - La Liga,2015/2016,Málaga,Sevilla,0,0,available,...,None,1,Regular Season,Estadio La Rosaleda,Alfonso Álvarez Izquierdo,Javier Gracia Carlos,Unai Emery Etxegoien,1.1.0,2,2


In [67]:
from statsbombpy import sb
events_df = sb.events(match_id=3754097)
events_df.head()

,50_50,ball_receipt_outcome,ball_recovery_offensive,ball_recovery_recovery_failure,block_deflection,carry_end_location,clearance_aerial_won,clearance_body_part,clearance_head,clearance_left_foot,...,substitution_outcome,substitution_outcome_id,substitution_replacement,substitution_replacement_id,tactics,team,team_id,timestamp,type,under_pressure
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"{'formation': 4231, 'lineup': [{'player': {'id...",Manchester United,39,00:00:00.000,Starting XI,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"{'formation': 4231, 'lineup': [{'player': {'id...",Tottenham Hotspur,38,00:00:00.000,Starting XI,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Manchester United,39,00:00:00.000,Half Start,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Tottenham Hotspur,38,00:00:00.000,Half Start,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Manchester United,39,00:00:00.000,Half Start,NaN


In [68]:
ordenado = events_df.sort_values(by='timestamp', ascending=False)
ordenado

,50_50,ball_receipt_outcome,ball_recovery_offensive,ball_recovery_recovery_failure,block_deflection,carry_end_location,clearance_aerial_won,clearance_body_part,clearance_head,clearance_left_foot,...,substitution_outcome,substitution_outcome_id,substitution_replacement,substitution_replacement_id,tactics,team,team_id,timestamp,type,under_pressure
3767,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Tottenham Hotspur,38,00:49:22.486,Half End,NaN
3766,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Manchester United,39,00:49:22.486,Half End,NaN
2089,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Tottenham Hotspur,38,00:49:21.634,Ball Receipt*,NaN
1089,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Tottenham Hotspur,38,00:49:18.897,Pass,NaN
2088,NaN,Incomplete,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Manchester United,39,00:49:07.135,Ball Receipt*,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"{'formation': 4231, 'lineup': [{'player': {'id...",Tottenham Hotspur,38,00:00:00.000,Starting XI,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Manchester United,39,00:00:00.000,Half Start,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Tottenham Hotspur,38,00:00:00.000,Half Start,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Manchester United,39,00:00:00.000,Half Start,NaN


In [69]:
ordenado[['minute', 'second', 'timestamp', 'type']]

,minute,second,timestamp,type
3767,94,22,00:49:22.486,Half End
3766,94,22,00:49:22.486,Half End
2089,94,21,00:49:21.634,Ball Receipt*
1089,94,18,00:49:18.897,Pass
2088,94,7,00:49:07.135,Ball Receipt*
...,...,...,...,...
1,0,0,00:00:00.000,Starting XI
4,45,0,00:00:00.000,Half Start
3,0,0,00:00:00.000,Half Start
2,0,0,00:00:00.000,Half Start


In [74]:
events_df[(events_df['team'] == 'Tottenham Hotspur') & (events_df['type'] == 'Shot')].shape

(9, 88)

In [ ]:
tiros.shape